# Matching expressions with spacy lookup

In [1]:
import spacy

In [2]:
sci_md = 'C:\\Users\\Sonja\\Anaconda3\\envs\\scispacy\\Lib\\site-packages\\en_core_sci_md\\en_core_sci_md-0.2.4'
nlp1 = spacy.load(sci_md, disable = ['ner']) #disabling ner required for Entity to work

In [3]:
virusterms = []
with open('corona_virus.txt','r') as viruslist:
    for line in viruslist:
        line = line.strip()
        virusterms.append(line)
print(virusterms)
viruslist.close()

['covid19 virus', 'covid 19 virus', 'sarscov2', 'sarscov2 2019', '2019 sarscov2', '2019novel sarscov2', '2019new sarscov2', '2019 novel sarscov2', '2019 new sarscov2', 'hcov 19', 'hcov19', 'sars cov 2', 'sars cov 2 2019', '2019 sars cov 2', '2019novel sars cov 2', '2019new sars cov 2', '2019 novel sars cov 2', '2019 new sars cov 2', 'new coronavirus', 'new coronavirus 2019', '2019 new coronavirus', 'new corona virus', 'new corona virus 2019', '2019 new corona virus', 'novel coronavirus', 'novel coronavirus 2019', '2019 novel coronavirus', 'novel corona virus', 'novel corona virus 2019', '2019 novel corona virus', 'wuhan virus', 'wuhan virus 2019', '2019 wuhan virus', '2019novel wuhan virus', '2019new wuhan virus', '2019 novel wuhan virus', '2019 new wuhan virus', 'wuhan pneumonia virus', 'wuhan pneumonia virus 2019', '2019 wuhan pneumonia virus', '2019novel wuhan pneumonia virus', '2019new wuhan pneumonia virus', '2019 novel wuhan pneumonia virus', '2019 new wuhan pneumonia virus', 'wu

In [4]:
from spacy_lookup import Entity

In [5]:
entity1 = Entity(keywords_list=virusterms,case_sensitive=False) #this merges tokens to match the keywords_list
nlp1.add_pipe(entity1, last=True)
print(nlp1.pipeline)

[('tagger', <spacy.pipeline.pipes.Tagger object at 0x000002AA45A6FC48>), ('parser', <spacy.pipeline.pipes.DependencyParser object at 0x000002AA45A73468>), ('entity', <spacy_lookup.Entity object at 0x000002AA50DDD848>)]


In [18]:
abstract = open('gold_abstracts.txt', encoding="utf8").read()

In [7]:
abstract = abstract.replace('-', '')
abstract = abstract.replace('/', ' ')
print(abstract)

1. Viruses. 2020 Jan 24;12(2). pii: E135. doi: 10.3390 v12020135.

Return of the Coronavirus: 2019nCoV.

Gralinski LE(1), Menachery VD(2)(3).

Author information: 
(1)Department of Epidemiology, Gillings School of Global Public Health,
University of North Carolina, Chapel Hill, NC 27514, USA.
(2)Department of Microbiology and Immunology, University of Texas Medical Branch,
Galveston, TX 77555, USA.
(3)Institute for Human Infections and Immunity, University of Texas Medical
Branch, Galveston, TX 77555, USA.

The emergence of a novel coronavirus (2019nCoV) has awakened the echoes of
SARSCoV from nearly two decades ago. Yet, with technological advances and
important lessons gained from previous outbreaks, perhaps the world is better
equipped to deal with the most recent emergent group 2B coronavirus.

DOI: 10.3390 v12020135 
PMID: 31991541


2. J Biol Regul Homeost Agents. 2020 Feb 4;34(1). doi: 10.23812 20EditorialKritas.
[Epub ahead of print]

Mast cells contribute to coronavirusinduced

In [19]:
doc1 = nlp1(abstract)

In [20]:
tokens = [token.text for token in doc1]
print(tokens)

['1', '.', 'Viruses', '.', '2020', 'Jan', '24;12(2', ')', '.', 'pii', ':', 'E135', '.', 'doi', ':', '10.3390/v12020135', '.', '\n\n', 'Return', 'of', 'the', 'Coronavirus', ':', '2019-nCoV.', '\n\n', 'Gralinski', 'LE(1', ')', ',', 'Menachery', 'VD(2)(3', ')', '.', '\n\n', 'Author', 'information', ':', '\n', '(1)Department', 'of', 'Epidemiology', ',', 'Gillings', 'School', 'of', 'Global', 'Public', 'Health', ',', '\n', 'University', 'of', 'North', 'Carolina', ',', 'Chapel', 'Hill', ',', 'NC', '27514', ',', 'USA', '.', '\n', '(2)Department', 'of', 'Microbiology', 'and', 'Immunology', ',', 'University', 'of', 'Texas', 'Medical', 'Branch', ',', '\n', 'Galveston', ',', 'TX', '77555', ',', 'USA', '.', '\n', '(3)Institute', 'for', 'Human', 'Infections', 'and', 'Immunity', ',', 'University', 'of', 'Texas', 'Medical', '\n', 'Branch', ',', 'Galveston', ',', 'TX', '77555', ',', 'USA', '.', '\n\n', 'The', 'emergence', 'of', 'a', 'novel coronavirus', '(', '2019-nCoV', ')', 'has', 'awakened', 'the', 

In [21]:
import pandas as pd
df_virus = pd.DataFrame()


for token in doc1:
    if token._.is_entity:
        token_text = token.text
        token_text = token.text.split()
        if len(token_text) == 1:
            token_series = {'token' : token.text, 'label' : "B", 'category' : 'Virus_SARS-CoV-2', 'sent': token.sent}
            df_virus = df_virus.append(token_series, ignore_index=True)
        else:
            token_series = {'token' : token_text[0], 'label' : "B", 'category' : 'Virus_SARS-CoV-2', 'sent': token.sent}
            df_virus = df_virus.append(token_series, ignore_index=True)
            index = 1
            while index <= (len(token_text)-1):
                token_series = {'token' : token_text[index], 'label' : "I", 'category' : 'Virus_SARS-CoV-2', 'sent': token.sent,}
                df_virus = df_virus.append(token_series, ignore_index=True)
                index = index+1
    else:
        token_series = {'token' : token.text, 'label' : "O", 'category' : 'NaN', 'sent': token.sent}
        df_virus = df_virus.append(token_series, ignore_index=True)
df_virus = df_virus[['token', 'label', 'category', 'sent']]

In [22]:
df_virus.to_csv (r'C:\Users\Sonja\gold_abstracts_virus1.csv', index = True, header=True)

In [12]:
nlp2 = spacy.load(sci_md, disable = ['ner']) #disabling ner required for Entity to work

In [13]:
diseaseterms = []
with open('C:\\Users\\Sonja\\corona_disease.txt','r') as diseaselist:
    for line in diseaselist:
        line = line.strip()
        diseaseterms.append(line)
print(diseaseterms)
diseaselist.close()

['covid19', 'covid 19', 'ncp', 'coronavirus disease 2019', 'corona virus disease 2019', 'coronavirus disease19', 'corona virus disease19', '2019 novel coronavirus respiratory syndrome', '2019 novel corona virus respiratory syndrome', 'wurs', 'seafood market pneumonia', 'severe acute respiratory syndrome type 2', 'sars 2', 'sars2', 'sars 2019', '2019 sars', 'severe acute respiratory syndrome 2019', 'wuhan infection', 'hubei infection', 'seafood market infection', 'covid19 virus infection', 'covid 19 virus infection', 'sarscov2 infection', 'sarscov2 2019 infection', '2019 sarscov2 infection', '2019novel sarscov2 infection', '2019new sarscov2 infection', '2019 novel sarscov2 infection', '2019 new sarscov2 infection', 'hcov 19 infection', 'hcov19 infection', 'sars cov 2 infection', 'sars cov 2 2019 infection', '2019 sars cov 2 infection', '2019novel sars cov 2 infection', '2019new sars cov 2 infection', '2019 novel sars cov 2 infection', '2019 new sars cov 2 infection', 'new coronavirus in

In [14]:
entity2 = Entity(keywords_list=diseaseterms,case_sensitive=False) #this merges tokens to match the keywords_list
nlp2.add_pipe(entity2, last=True)
print(nlp2.pipeline)

[('tagger', <spacy.pipeline.pipes.Tagger object at 0x000002AA6130E988>), ('parser', <spacy.pipeline.pipes.DependencyParser object at 0x000002AA61309828>), ('entity', <spacy_lookup.Entity object at 0x000002AA40A60808>)]


In [16]:
doc2 = nlp2(abstract)

In [17]:
df_disease = pd.DataFrame()


for token in doc2:
    if token._.is_entity:
        token_text = token.text
        token_text = token.text.split()
        if len(token_text) == 1:
            token_series = {'token' : token.text, 'label' : "B", 'category' : 'Disease_COVID-19', 'sent': token.sent}
            df_disease = df_disease.append(token_series, ignore_index=True)
        else:
            token_series = {'token' : token_text[0], 'label' : "B", 'category' : 'Disease_COVID-19', 'sent': token.sent}
            df_disease = df_disease.append(token_series, ignore_index=True)
            index = 1
            while index <= (len(token_text)-1):
                token_series = {'token' : token_text[index], 'label' : "I", 'category' : 'Disease_COVID-19', 'sent': token.sent,}
                df_disease = df_disease.append(token_series, ignore_index=True)
                index = index+1
    else:
        token_series = {'token' : token.text, 'label' : "O", 'category' : 'NaN', 'sent': token.sent}
        df_disease = df_disease.append(token_series, ignore_index=True)
df_disease = df_disease[['token', 'label', 'category', 'sent']]
df_disease.to_csv (r'C:\Users\Sonja\gold_abstracts_disease.csv', index = True, header=True)